Este notebook lo usamos para sacar los datos de los recorridos que (por desgracia) se tienen que sacar a mano.

Cosas que pongo aquí porque si no se me olvidan:

* Los carriles se enumeran desde 0 (el derecho) hasta n (el más izquiero de todos). Eso quiere decir que los cambios son -1 a la derecha y +1 a la izquierda.
* Si el siguiente semáforo está muy lejos, nos da exactamente igual el estado en el que esté. Este hecho lo podemos usar para generar más datos.
* La precisión del GPS es un poco chusta, así que los resultados mejorarían dramáticamente en el caso de conseguir una mayor precisión en la toma de datos.

In [ ]:
%matplotlib notebook

import os, glob
from functools import partial

import pandas as pd
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pynsia.pointcloud import PointCloud, transformation_matrix
from sklearn.cluster import DBSCAN

import ipywidgets as widgets
from IPython.display import display

from pynsia import latlon

from utils import load_master_df, load_subject_df, DATASETS_INFO


figsize = (8, 4)

In [ ]:
# BASE_PATH = '/home/blazaid/Projects/data-phd/sync'
BASE_PATH = '/media/blazaid/Saca/Phd/data/sync'
OUTPUT_PATH = '/media/blazaid/Saca/Phd/data/curated'
SUBJECT = 'miguel'  # edgar, jj, miguel
DATASET = 'validation'  # training, validation
MAX_LEADER_DISTANCE = 50
MAX_TLS_DISTANCE = 100
MAX_DRIVABLE_DISTANCE = 100

In [ ]:
# Load the subject's data
df = load_subject_df(BASE_PATH, SUBJECT, DATASET, 'dataset')
# Select user info
dataset_info = DATASETS_INFO[SUBJECT][DATASET]
# Subject's path
SUBJECT_PATH = os.path.join(BASE_PATH, SUBJECT, DATASET)
# Calibration data
calibration_data = dataset_info['calibration_data']

## Subconjuntos de entrenamiento y test

De los conjuntos sincronizados originales extraeremos unos subconjuntos similares para trabajar con ellos, de tal manera que para cada sujeto los comienzos y los finales de las rutas sean aproximadamente los mismos.

Estos datos se han sacado visualizando los vídeos de los sujetos.

In [ ]:
starting_frame = dataset_info['starting_frame'] or 0
ending_frame = dataset_info['ending_frame'] or (len(df) - 1)

print('Starting frame:\t{}'.format(starting_frame))
print('Ending frame:\t{}'.format(ending_frame))
print('New route len:\t{}'.format(ending_frame + 1 - starting_frame))
df = df[starting_frame:ending_frame + 1].reset_index(drop=True)

## Cambios de carril

El cálculo de los cambios de carril (desgraciadamente) se hace a mano. Está definido en la variable DATASETS_INFO para cada conductor y dataset, y se ha obtenido mediante la visualización de los vídeos.

In [ ]:
df['Lane change'] = 0
for ini_frame, end_frame, change in dataset_info['lane_changes']:
    df.loc[ini_frame:end_frame, 'Lane change'] = change
fig, ax = plt.subplots(1, 1, figsize=figsize)
df['Lane change'].plot(ax=ax);

## Velocidad máxima

Como en los cambios de carril, la velocidad máxima se ha extraído manualmente a partir de la información de los mapas de openstreetmap y de los vídeos de los recorridos.

In [ ]:
df['Max speed'] = 0
for frame, speed in dataset_info['max_speed']:
    df.loc[frame:, 'Max speed'] = speed
fig, ax = plt.subplots(1, 1, figsize=figsize)
df['Max speed'].plot(ax=ax);

## Distancias

Todas las distancias se calcularán más o menos de la misma manera (salvo la del vehículo delantero). Se tiene una tabla maestra con las posiciones de los puntos de interes contra los que calcular la distancia. A partir de ahí se generan tantas columnas como puntos de interes y tantas filas como filas tiene el recorrido. Después, de forma manual, se irá eligiendo qué punto de interes corresponde a qué intervalo y se generará la nueva tabla con dichos valores.

### Distancias a semáforos

Cada sujeto tiene un fichero con la información de cuál es el siguiente semáforo para cada distancia. Se generará por tanto una columna con las distancias al siguiente semáfolo y su estado.

In [ ]:
# Base TLS dataframe
master_tls_df = load_master_df(BASE_PATH, DATASET, 'tls')
master_tls_df = master_tls_df.set_index('tls')
# Subject tls's dataframe
tls_df = load_subject_df(BASE_PATH, SUBJECT, DATASET, 'tls')

In [ ]:
df['Next TLS distance'] = 0.0
df['Next TLS status'] = 0.0
for i, row in tls_df.iterrows():
    next_tls = row['next_tls']
    
    lat, lon = master_tls_df.loc[row['next_tls'],['lat', 'lon']]
    df.loc[row['frame']:,'Next TLS status'] = row['status']
    df.loc[row['frame']:,'Next TLS distance'] = df.loc[row['frame']:,['gps_positions_latitude', 'gps_positions_longitude']].apply(lambda r: latlon.distance(
        (r['gps_positions_latitude'], r['gps_positions_longitude']),
        (lat, lon)
    ), axis=1)

fig, ax = plt.subplots(1, 1, figsize=figsize)
df[['Next TLS distance']].plot(ax=ax);

### Distancias recorribles

En cada momento, el vehículo tendrá noción de cuánta distancia puede recorrer en su carril, el carril de la izquierda y el de la derecha. Se pretende de esta manera que el modelo sea capaz de determinar cambios de carril. Esta información está disponible en SUMO, razón por la que se ha intentado obtener aquí. Ha requerido de bastante trabajo manual.

In [ ]:
df['Distance +1'] = df['Distance 0'] = df['Distance -1'] = np.inf

for frame, left, current, right in dataset_info['lanes_distances']:
    # Is the same code for each column, so we do this for over all of them
    for lane, column in zip([left, current, right], ['Distance +1', 'Distance 0', 'Distance -1']):
        # Three options, inf (default), tuple (calc. distance) or 0
        if lane != np.inf:
            if isinstance(lane, tuple):
                lat, lon = lane
                df.loc[frame:,column] = df.loc[frame:,['gps_positions_latitude', 'gps_positions_longitude']].apply(
                    lambda r: latlon.distance(
                        (r['gps_positions_latitude'], r['gps_positions_longitude']),
                        (lat, lon)
                    ), axis=1
                )
            else:
                df.loc[frame:,column] = 0
fig, ax = plt.subplots(1, 1, figsize=figsize)
np.minimum(50, df[['Distance +1', 'Distance 0', 'Distance -1']]).plot(ax=ax)

### Distancia al objeto más cercano

Para sacar la distancia al objeto delantero, se usará una porción rectangular de la nube de puntos para detectar clústers y así determinar si el objeto delantero es un obstáculo.

Requiere trabajo manual, pero al menos es mejor que nada.

In [ ]:
df['Leader distance'] = np.nan
for frame_ini, frame_end, epsilon, min_samples in dataset_info['cf_dist']:
    for frame, row in df[frame_ini:frame_end].iterrows():
        pointcloud_path = row['pointclouds_path']
        if not pd.isnull(pointcloud_path):
            ps = PointCloud.load(pointcloud_path).transform(**calibration_data).points

            # Extract the points in the specified bounding box
            mask = (ps[:,2] > -1.5) & (ps[:,2] < 0.5) & (ps[:,0] < 35) & (ps[:,0] > 0.35) & (ps[:,1] < 1) & (ps[:,1] > -1)
            masked_points = ps[mask,:]

            # Look for clusters and extract the distance to them
            n_clusters = 0
            n_points = len(masked_points)
            dist = np.nan
            if len(masked_points) > 0:
                db = DBSCAN(eps=epsilon, min_samples=min_samples).fit(masked_points)
                labels = db.labels_
                n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
                for cluster in range(n_clusters):
                    obs = masked_points[labels == cluster]
                    ax.scatter(obs[:,0], obs[:,1], s=1, color='r', alpha=0.1);
                    centroid_dist = np.sqrt(np.mean(obs[:, 0])**2 + np.mean(obs[:, 1])**2)
                    dist = centroid_dist if pd.isnull(dist) else min(dist, centroid_dist)

            df.loc[frame, 'Leader distance'] = dist
    
    df.loc[frame_ini:frame_end,'Leader distance'] = df.loc[frame_ini:frame_end,'Leader distance'].interpolate()


fig, ax = plt.subplots(1, 1, figsize=figsize)
df['Leader distance'].plot(ax=ax)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=figsize)
df[['canbus_Speed (m/s)']] = df[['canbus_Speed (km/h)']] * 1000 / 3600
df['canbus_Speed (m/s)'].plot(ax=ax)
df['gps_speeds_speed'].plot(ax=ax)
df['gps_speeds_speed'].rolling(5, center=True).mean().plot(ax=ax)

## Extracción de los datasets

In [ ]:
if not os.path.isdir(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

### Funciones de transformación de datos

Algunas funciones serán comunes para transformar los datos. Por tanto, las definimos antes.

#### Estado del semáforo

Los semáforos tienen un estado de _g_, _y_ o _r_, en función de si están en verde, amarillo o rojo. se transformarán a un valor numérico 0, 0.5 y 1 respectivamente.

In [ ]:
def status_to_number(status):
    return {'g': 0, 'y': 0.5, 'r': 1}[status.lower()]

#### Distancias relativas

Las distancias relativas se definirán de acuerdo a un máximo. A partir de ahí, se normalizarán de 0 a 1 siendo 0 la mínima distancia y 1 cualquier valor mayor o igual a la máxima distancia definida.

In [ ]:
def relative_bounded(value, maximum):
    return min(1, value / maximum)

### Datasets para entrenamiento de controladores difusos

Los dataframes para el entrenamiento de controladores difusos se separarán en varias secuencias, una por cada tramo de análisis. Luego, desde ahí, se calcularán el resto de columnas para alimentar al modelo.

Las columnas calculadas con las que se alimentará el modelo son las siguientes:

* **Acceleration**: The acceleration to apply, in m/s².
* **Leader distance**: La distancia al vehículo que está por delante. El valor estará normalizado al intervalo $[0, 1]$ donde 0 se corresponderá a 0 m/s y 1 a la máxima distancia considerada para el experimento (50 metros o más).
* **Next TLS distance**: Distancia relativa al próximo semáforo. El valor estará normalizado al intervalo $[0, 1]$ donde 0 se corresponderá a 0 m/s y 1 a la máxima distancia considerada para el experimento (50 metros o más).
* **Next TLS status**: El estado del siguiente semáforo, que será 0, 0.5 y 1 en función de si se encuentra en verde, amarillo o rojo.
* **Relative speed**: La velocidad relativa en función de la velocidad máxima de la vía. El valor estará comprendido en el intervalo $[0, \infty)$, siendo $1$ la velocidad correspondiente a la velocidad máxima de la vía.
* **Speed to leader**: La velocidad relativa de aproximación al vehículo delantero.

In [ ]:
cf_sequences = []

for frame_ini, frame_end, _, _ in dataset_info['cf_dist']:
    sequence = df[frame_ini:frame_end].copy().reset_index(drop=True)

    max_speed = sequence['Max speed'] * 10 / 36  # km/h a m/s
    
    leader_distance = sequence['Leader distance'].apply(lambda x: relative_bounded(x, MAX_LEADER_DISTANCE))
    next_tls_distance = sequence['Next TLS distance'].apply(lambda x: relative_bounded(x, MAX_TLS_DISTANCE))
    next_tls_status = sequence['Next TLS status'].apply(status_to_number)
    relative_speed = sequence['gps_speeds_speed'].rolling(5, center=True).mean() / max_speed
    speed_to_leader = (sequence['Leader distance'] - sequence['Leader distance'].shift(1))
    acceleration = relative_speed.shift(-1) - relative_speed

    car_following_df = pd.DataFrame({
        'Leader distance': leader_distance,
        'Next TLS distance': next_tls_distance,
        'Next TLS status': next_tls_status,
        'Relative speed': relative_speed,
        'Speed to leader': speed_to_leader,
        'Acceleration': acceleration
    })[2:-2].reset_index(drop=True)
    
    cf_sequences.append(car_following_df)

In [ ]:
filename_prefix = 'cf_{}_{}'.format(SUBJECT, DATASET)
print('Deleting previous cf sequences ({}, {})'.format(SUBJECT, DATASET))
for filename in glob.glob(os.path.join(OUTPUT_PATH, filename_prefix + '*')):
    print('\t' + filename)
    os.remove(filename)
    
print('Saving new sequences')
for i, sequence in enumerate(cf_sequences):
    filename = '{}_{:0>5}.csv'.format(filename_prefix, i)
    output_path = os.path.join(OUTPUT_PATH, filename)
    print('\t' + output_path)
    sequence.to_csv(output_path, index=False)

### Datasets para entrenamiento de redes de convolución

Este dataset es algo más complejo, ya que involucra más columnas, imágenes, transformaciones, _data augmentation_, etcétera. Primero obtendremos todas aquellas secuencias con continuidad de nube de puntos y de más 2 segundos o más, esto es, todas aquellas secuencias de 20 o más frames sin un NaN en la celda de pointcloud.

In [ ]:
temp_lc_sequences = []
frame_ini = 0
for frame_end in pd.isnull(df['pointclouds_path']).nonzero()[0]:
    sequence = df[frame_ini:frame_end]
    if len(sequence) > 20:
        temp_lc_sequences.append(sequence)
    frame_ini = frame_end + 1

In [ ]:
temp_lc_sequences[0].columns

Ahora transformaremos las columnas de estas secuencias para que sean las siguientes:

* **Acceleration**: The acceleration to apply, in m/s².
* **Distance +1, 0 y -1**: Las distancias (relativas al máximo de distancias) que se pueden recorrer en ellos carriles izquierdo, aztual y derecho.
* **Lane change**. Cuándo está ocurriendo un cambio de carril. será 0, 1 o 2 en función de si el cambio es a la izquierda, no hay cambio, o a la derecha.
* **Next TLS distance**: Distancia relativa al próximo semáforo. El valor estará normalizado al intervalo $[0, 1]$ donde 0 se corresponderá a 0 m/s y 1 a la máxima distancia considerada para el experimento (50 metros o más).
* **Next TLS status**: El estado del siguiente semáforo, que será 0, 0.5 y 1 en función de si se encuentra en verde, amarillo o rojo.
* **Point cloud**. El path al fichero con la nube de puntos.
* **Relative speed**: La velocidad relativa en función de la velocidad máxima de la vía. El valor estará comprendido en el intervalo $[0, \infty)$, siendo $1$ la velocidad correspondiente a la velocidad máxima de la vía.
* **Speed**: La velocidad relativa en función de la velocidad máxima de la vía. El valor estará comprendido en el intervalo $[0, \infty)$, siendo $1$ la velocidad correspondiente a la velocidad máxima de la vía.

In [ ]:
lc_master_sequences = []
for sequence in temp_lc_sequences:
    max_speed = sequence['Max speed'] * 10 / 36  # km/h a m/s
    
    relative_speed = sequence['gps_speeds_speed'].rolling(5, center=True).mean() / max_speed
    distance_l_lane = sequence['Distance +1'].apply(lambda x: relative_bounded(x, MAX_DRIVABLE_DISTANCE))
    distance_c_lane = sequence['Distance 0'].apply(lambda x: relative_bounded(x, MAX_DRIVABLE_DISTANCE))
    distance_r_lane = sequence['Distance -1'].apply(lambda x: relative_bounded(x, MAX_DRIVABLE_DISTANCE))
    acceleration = relative_speed.shift(-1) - relative_speed
    pointcloud_path = sequence['pointclouds_path']
    lane_change = sequence['Lane change'] + 1
    next_tls_distance = sequence['Next TLS distance'].apply(lambda x: relative_bounded(x, MAX_TLS_DISTANCE))
    next_tls_status = sequence['Next TLS status'].apply(status_to_number)
    speed_to_leader = (sequence['Leader distance'] - sequence['Leader distance'].shift(1))

    lane_change_df = pd.DataFrame({
        'Acceleration': acceleration,
        'Distance +1': distance_l_lane,
        'Distance 0': distance_c_lane,
        'Distance -1': distance_r_lane,
        'Lane change': lane_change,
        'Relative speed': relative_speed,
        'Pointcloud': pointcloud_path,
        'Next TLS distance': next_tls_distance,
        'Next TLS status': next_tls_status,
        'Speed to leader': speed_to_leader,
    })[2:-2].reset_index(drop=True)
    
    lc_master_sequences.append(lane_change_df)
lc_master_sequences[0]